In [1]:
# Installs geemap package
import subprocess
import os
import pandas as pd
import geopandas as gpd
import json #do I need this if I'm using geopandas?
import rasterio as rst

import geemap
import ee

In [2]:
import ee
import geemap

In [3]:
print(os.getcwd())
#print(os.listdir('.'))

/home/jennaguffogg/workspace/sensand/repos/jenna-test


In [4]:
#ee.Authenticate()
ee.Initialize()

geom_gpd = gpd.read_file('marysville-fire-forest-loss-test-area.geojson')
geom = geemap.geopandas_to_ee(geom_gpd)


In [5]:
#get dynamic world data
START = '2021-04-01'
END = '2021-04-10'

col_filter = ee.Filter.And(
    ee.Filter.bounds(geom),
    ee.Filter.date(START, END),
)

dw_col = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filter(col_filter)
s2_col = ee.ImageCollection('COPERNICUS/S2').filter(col_filter)

# Join corresponding DW and S2 images (by system:index).
dw_s2_col = ee.Join.saveFirst('s2_img').apply(
    dw_col,
    s2_col,
    ee.Filter.equals(leftField='system:index', rightField='system:index'),
)

# Extract an example DW image and its source S2 image.
dw_image = ee.Image(dw_s2_col.first())
s2_image = ee.Image(dw_image.get('s2_img'))



In [6]:
# Create a visualization that blends DW class label with probability.
# Define list pairs of DW LULC label and color.
CLASS_NAMES = [
    'water',
    'trees',
    'grass',
    'flooded_vegetation',
    'crops',
    'shrub_and_scrub',
    'built',
    'bare',
    'snow_and_ice',
]

VIS_PALETTE = [
    '419bdf',
    '397d49',
    '88b053',
    '7a87c6',
    'e49635',
    'dfc35a',
    'c4281b',
    'a59b8f',
    'b39fe1',
]

# Create an RGB image of the label (most likely class) on [0, 1].
dw_rgb = (
    dw_image.select('label')
    .visualize(min=0, max=8, palette=VIS_PALETTE)
    .divide(255)
)

# Get the most likely class probability.
top1_prob = dw_image.select(CLASS_NAMES).reduce(ee.Reducer.max())

# Create a hillshade of the most likely class probability on [0, 1]
top1_prob_hillshade = ee.Terrain.hillshade(top1_prob.multiply(100)).divide(255)

# Combine the RGB image with the hillshade.
dw_rgb_hillshade = dw_rgb.multiply(top1_prob_hillshade)



In [7]:
m = geemap.Map()
m.add_basemap('HYBRID')
m.addLayer(geom, {})
m.centerObject(geom)
m.add_layer(
    s2_image,
    {'min': 0, 'max': 3000, 'bands': ['B4', 'B3', 'B2']},
    'Sentinel-2 L1C',
)
m.add_layer(
    dw_rgb_hillshade,
    {'min': 0, 'max': 0.65},
    'Dynamic World V1 - label hillshade',
)
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [10]:
# Create a Sentinel-2 image composite
clip_mask = geom
image = geemap.dynamic_world_s2(geom, START, END, clip=True)
vis_params = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}

m2 = geemap.Map()
m2.add_basemap('HYBRID')

m2.addLayer(geom, {})
m2.centerObject(geom)

m2.addLayer(image, vis_params, 'Sentinel-2 image')

# Create Dynamic World land cover composite
landcover = geemap.dynamic_world(geom, START, END, clip=True, return_type='hillshade')
m2.addLayer(landcover, {}, 'Land Cover')

# Add legend to the map
m2.add_legend(title="Dynamic World Land Cover", builtin_legend='Dynamic_World')
m2

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [9]:
# Save Dynamic World class data in GeoTIFF format
#output_path = 'landcover.tif'
#landcover = geemap.dynamic_world(region, start_date, end_date, return_type='class')
#geemap.ee_export_image(landcover, filename=output_path, scale=10, region=region, file_per_band=False)